**⭐ 1. What This Pattern Solves**

Streaming aggregations allow you to compute metrics continuously on a stream of data rather than batch.
Use-cases include:

Real-time sales per product

Active users per minute/hour

Error counts from logs in real-time

The main challenge is maintaining aggregates efficiently over unbounded streams.

**⭐ 2. SQL Equivalent**

In [0]:
%sql
SELECT product_id, SUM(sales_amount) AS total_sales, window(event_time, '1 hour') AS hour_window
FROM sales_stream
GROUP BY product_id, window(event_time, '1 hour')

**⭐ 3. Core Idea**

Use structured streaming with groupBy on keys + window

Aggregations are incremental; only changes since last trigger are processed

Works with event-time windows for time-based metrics

Supports watermarks to handle late data

**⭐ 4. Template Code (MEMORIZE THIS)**

In [0]:
from pyspark.sql.functions import window, sum as Fsum

streaming_df.groupBy(
    "key_col",
    window("event_time", "window_duration")
).agg(
    Fsum("metric_col").alias("agg_metric")
)
.writeStream \
    .outputMode("update") \
    .format("console") \
    .start()


**⭐ 5. Detailed Example**

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, sum as Fsum
import pyspark.sql.types as T

spark = SparkSession.builder.getOrCreate()

# Simulate streaming source
schema = T.StructType([
    T.StructField("product_id", T.StringType()),
    T.StructField("sales_amount", T.IntegerType()),
    T.StructField("event_time", T.TimestampType())
])

streaming_df = spark.readStream.schema(schema).json("/tmp/sales_stream")

agg_df = streaming_df.groupBy(
    "product_id",
    window("event_time", "1 hour")
).agg(
    Fsum("sales_amount").alias("total_sales")
)

query = agg_df.writeStream.outputMode("update").format("console").start()
query.awaitTermination()


**Step-by-step:**

Read stream and define schema

Group by key and event-time window

Aggregate metric incrementally

Output continuously

**⭐ 6. Mini Practice Problems**

Count user logins per 5-minute window.

Compute average order value per 1-hour window.

Find top 3 products per 30-minute window in a streaming source.

**⭐ 7. Full Data Engineering Problem**

Scenario: E-commerce platform needs real-time dashboard of sales per product per hour. Data arrives via Kafka (millions of events/day).

Solution Approach:

Stream data from Kafka

Assign event time and watermark

Aggregate using groupBy + window

Write to Delta table or dashboard

Optimize: watermarking, stateful aggregation, trigger intervals

**⭐ 8. Time & Space Complexity**

Time: O(N) per micro-batch, dependent on aggregation keys and window size

Space: O(unique keys × window state)

Large key cardinality → consider state cleanup & watermarks

**⭐ 9. Common Pitfalls**

Forgetting to use watermark → state grows unbounded

Using append mode with non-time windows → results missing

Large window + high cardinality → memory overflow

Incorrect event-time assignment → late events ignored